In [4]:
import os
from PIL import Image
import pytesseract
import re
import cv2
import matplotlib.pyplot as plt
import pandas as pd

image_folder = "images"  # Path to the folder containing images

# List all files in the folder
image_files = os.listdir(image_folder)

# Loop through the image files and display them
for file_name in image_files[:3]:
    if file_name.endswith((".jpg", ".png")):  # Filter for image files
        img_path = os.path.join(image_folder, file_name)
        img = Image.open(img_path)
        img.show()  # Display the image

In [4]:
# Define the path to the images
image_folder = 'images'  # Replace with your image folder path

# List all files in the folder
image_files = os.listdir(image_folder)


In [5]:
'''
# Folder containing images
image_folder = 'images/'  # Replace with the path to your images folder

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png", ".jpeg"))]'''

In [10]:
def preprocess_image(image_path):
    # Load the image with OpenCV
    img = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow(gray)

    # Increase the contrast of the image
    contrast = cv2.convertScaleAbs(gray, alpha=2, beta=0)

    # Use adaptive thresholding for varying light conditions
    thresh = cv2.adaptiveThreshold(contrast, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY, 11, 2)

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(thresh, (5, 5), 0)

    # Resize the image to improve OCR accuracy (double the size)
    resized = cv2.resize(blurred, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    # Return preprocessed image
    return resized, img, blurred


In [12]:
def test_text_detection(image_path):
    # Load the image
    img = cv2.imread(image_path)
    
    (h, w) = img.shape[:2]
    cropped_img = img[:, w // 2:]

    # Extract all text from the image
    detected_text = pytesseract.image_to_string(cropped_img)
    
    # Print the detected text
    print("Detected Text:")
    print(detected_text)

# Example Usage
img_folder = 'images/'  # Path to your images folder
image_files = [f for f in os.listdir(img_folder) if f.endswith((".jpg", ".png", ".jpeg"))]

for file_name in image_files[:2]:
    img_path = os.path.join(img_folder, file_name)
    print(f"Testing text detection for {file_name}...")
    test_text_detection(img_path)


Testing text detection for 0.jpg...
Detected Text:
MART

OW PRICES.
une
RCENTER
24 HOURS
FR . TBA

986 = 175d
0158 Ae TR# 03178
00004
b /0.49 0.20 N
00010451 F 5.48 N
UNT GIVEN 0.57
SUBTOTAL Sid

TOTAL Se

ASH TEND 11.00
HANGE DUE 5.89
9
> SOLD 2

iin


Testing text detection for 1.jpg...
Detected Text:
i JOE’S

le Ave
15206
) 334-0614
OOPM DAILY
1.29
1.99
1.59
BASIL 1.59
TEAL 2.69
2.49
LITET 3.99
Ns Bae
0.89
Pee)
eee)
Rel
ED 0.99
0.98
0.87
UTTER 2.49
1.69
1,38

$38 .68
$38 .68
$40.00

$1.32

Higgins, Ryan
0403 04 1346 4683

SHOPPING AT
2 JOE'S
2rjoes .com



In [17]:
def load_images(folder_path):
    """
    Load all images from the given folder and yield each image.
    :param folder_path: Path to the folder containing images.
    :return: A generator that yields images one by one.
    """
    # List all files in the folder
    for filename in os.listdir(folder_path):
        # Only consider image files (you can add more formats if needed)
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            image_path = os.path.join(folder_path, filename)
            img = cv2.imread(image_path)  # Read the image
            (h, w) = img.shape[:2]
            cropped_img = img[:, w // 2:]

            if cropped_img is not None:
                yield cropped_img, filename  # Yield the image and its filename

In [ ]:
res=[]
dic={}
# Iterate through images in the folder
for img, filename in load_images('images'):
    print(f"Processing image: {filename}")
    
    # Use pytesseract to extract OCR data
    d = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)

    # Price pattern regex to match prices
    price_pattern = r'(\$?[\d,]+\.\d{2})'

    print(f"{'Text':<12} {'X':<5} {'Y':<5} ")
    # Iterate through each OCR result
    for i in range(len(d['text'])):
        text = d['text'][i].strip()  # Remove leading/trailing spaces
        if text:  # Skip empty text
            match = re.search(price_pattern, text)  # Search for price pattern
            if match:
                x, y=d['left'][i], d['top'][i]
                price=match.group(0)
                res.append(price)
                dic[price]=y
                print(f"{match.group(0): <12} {x:<5} {y:<5}")
print(res)
print(dic)



Processing image: 0.jpg
Text         X     Y     
0.49         23    497  
0.20         176   499  
5.48         176   521  
0.57         174   564  
11.00        162   624  
5.89         172   649  
Processing image: 1.jpg
Text         X     Y     
1.29         87    325  
1.99         88    345  
1.59         88    365  
1.59         89    384  
2.69         88    403  
2.49         88    422  
3.99         89    441  
0.89         90    476  
0.99         95    550  
0.98         97    569  
0.87         100   607  
2.49         104   645  
1.69         107   665  
$40.00       132   785  
$1.32        142   805  
Processing image: 10.jpg
Text         X     Y     
Processing image: 11.jpg
Text         X     Y     
6.99         115   261  
3.69         117   280  
1.29         119   301  
2.99         119   321  
2.99         120   342  
3.49         121   364  
8.99         122   386  
2.00         124   409  
2.69         125   432  
2.15         128   480  
3.26         139   638 

In [28]:
import pandas as pd
#dataframe of prices with its bbox y-coordinate
df=pd.DataFrame(list(dic.items()), columns=['Text', 'Y-Co-ordinate'])
print(df)

      Text  Y-Co-ordinate
0     0.49            179
1     0.20            499
2     5.48            521
3     0.57            564
4    11.00            624
..     ...            ...
74  $28.28           1233
75   89.13            526
76    0.55            593
77    2.91            614
78    3.52            637

[79 rows x 2 columns]


In [ ]:
for 